In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"
 
schema_silver = "silver"
schema_gold = "gold"
schema_gold = "auditoria"

In [0]:
%sql
CREATE OR REPLACE VIEW dmc_tallerfinaldatabricks_mariovento.gold.kpi_ticket_promedio_tienda_mes AS
SELECT 
    t.sk_tienda,
    dt.anio,
    dt.mes,
    ROUND(SUM(f.monto) / NULLIF(SUM(f.cantidad), 0), 2) AS ticket_promedio
FROM dmc_tallerfinaldatabricks_mariovento.gold.fact_ventas f
JOIN dmc_tallerfinaldatabricks_mariovento.gold.dim_tienda t ON f.sk_tienda = t.sk_tienda
JOIN dmc_tallerfinaldatabricks_mariovento.gold.dim_tiempo dt ON f.sk_tiempo = dt.sk_tiempo
GROUP BY t.sk_tienda, dt.anio, dt.mes;

In [0]:
%sql
SELECT * FROM dmc_tallerfinaldatabricks_mariovento.gold.kpi_ticket_promedio_tienda_mes;

In [0]:
%sql
CREATE OR REPLACE VIEW dmc_tallerfinaldatabricks_mariovento.gold.kpi_clientes_nuevos_mes AS
WITH primeras_compras AS (
    SELECT 
        f.sk_cliente,
        MIN(f.fecha_venta) AS primera_compra
    FROM dmc_tallerfinaldatabricks_mariovento.gold.fact_ventas f
    GROUP BY f.sk_cliente
)
SELECT 
    dt.anio,
    dt.mes,
    COUNT(pc.sk_cliente) AS clientes_nuevos
FROM primeras_compras pc
JOIN dmc_tallerfinaldatabricks_mariovento.gold.dim_tiempo dt ON TO_DATE(pc.primera_compra) = dt.fecha
GROUP BY dt.anio, dt.mes;

In [0]:
%sql
SELECT * FROM dmc_tallerfinaldatabricks_mariovento.gold.kpi_clientes_nuevos_mes;

In [0]:
%sql
CREATE OR REPLACE VIEW dmc_tallerfinaldatabricks_mariovento.gold.kpi_tasa_devolucion AS
SELECT 
    dt.anio,
    dt.mes,
    COUNT(DISTINCT d.id_venta) * 1.0 / COUNT(DISTINCT f.id_venta) AS tasa_devolucion
FROM dmc_tallerfinaldatabricks_mariovento.gold.fact_ventas f
JOIN dmc_tallerfinaldatabricks_mariovento.gold.dim_tiempo dt ON f.sk_tiempo = dt.sk_tiempo
LEFT JOIN dmc_tallerfinaldatabricks_mariovento.silver.devoluciones d ON f.id_venta = d.id_venta
GROUP BY dt.anio, dt.mes;

In [0]:
%sql
SELECT * FROM dmc_tallerfinaldatabricks_mariovento.gold.kpi_tasa_devolucion;

In [0]:
%sql
CREATE OR REPLACE VIEW dmc_tallerfinaldatabricks_mariovento.gold.kpi_crecimiento_mom_categoria AS
WITH ventas_mes AS (
    SELECT 
        p.categoria,
        dt.anio,
        dt.mes,
        SUM(f.monto) AS ventas_mes
    FROM dmc_tallerfinaldatabricks_mariovento.gold.fact_ventas f
    JOIN dmc_tallerfinaldatabricks_mariovento.gold.dim_producto p ON f.sk_producto = p.sk_producto
    JOIN dmc_tallerfinaldatabricks_mariovento.gold.dim_tiempo dt ON f.sk_tiempo = dt.sk_tiempo
    GROUP BY p.categoria, dt.anio, dt.mes
)
SELECT 
    categoria,
    anio,
    mes,
    ventas_mes,
    LAG(ventas_mes) OVER (PARTITION BY categoria ORDER BY anio, mes) AS ventas_mes_anterior,
    ROUND(
        (ventas_mes - LAG(ventas_mes) OVER (PARTITION BY categoria ORDER BY anio, mes)) 
        / NULLIF(LAG(ventas_mes) OVER (PARTITION BY categoria ORDER BY anio, mes),0), 4
    ) AS crecimiento_mom
FROM ventas_mes;

In [0]:
%sql
SELECT * FROM dmc_tallerfinaldatabricks_mariovento.gold.kpi_crecimiento_mom_categoria;

In [0]:
%sql
CREATE OR REPLACE VIEW dmc_tallerfinaldatabricks_mariovento.gold.kpi_resumen_incremental AS
SELECT 
    job_id,
    job_run_id,
    task_run_id,
    job_start_time,
    job_end_time,
    rows_inserted,
    rows_updated,
    rows_deleted,
    merge_duration_seconds
FROM dmc_tallerfinaldatabricks_mariovento.auditoria.ingestion_log
WHERE job_start_time = (
    SELECT MAX(job_start_time) 
    FROM dmc_tallerfinaldatabricks_mariovento.auditoria.ingestion_log
);

In [0]:
%sql
SELECT * FROM dmc_tallerfinaldatabricks_mariovento.gold.kpi_resumen_incremental;